In [1]:
from langchain.agents import initialize_agent, Tool, load_tools, AgentType
from langchain.tools import BaseTool
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.llms import VertexAI
from langchain_community.utilities import GoogleSearchAPIWrapper
from vertexai.language_models import TextGenerationModel
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import vertexai
import os
import time
import pandas as pd
import mapply
import json
mapply.init(
    n_workers=-1,
    chunk_size=1000,
    max_chunks_per_worker=0,
    progressbar=True
)

/home/user/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


llm = VertexAI(
    model_name='text-bison@001',
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

/home/user/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.vertexai.VertexAI` was deprecated in langchain-community 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAI`.
  warn_deprecated(


In [8]:
with open('keys.json') as f:
    keys = json.load(f)


In [21]:
os.environ["GOOGLE_CSE_ID"] = keys["GOOGLE_CSE_ID"]
os.environ["GOOGLE_API_KEY"] = keys["GOOGLE_API_KEY"]


NameError: name 'keys' is not defined

In [4]:
search = GoogleSearchAPIWrapper(k=1)

tool = [Tool(name="Google Search",
             description="Search Google for recent results.",
             func=search.run,
             )
        ]

In [5]:
agent = initialize_agent(tool, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

/home/user/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [31]:
def fetch_parent_company(brand):
    result = agent.run(f"What is the parent company of {brand}")
    time.sleep(1)
    return result

In [53]:
engine = create_engine('bigquery://us-gcp-ame-con-52dbb-sbx-1')

/tmp/ipykernel_1172/111781775.py:1: SADeprecationWarning: The dbapi() classmethod on dialect classes has been renamed to import_dbapi().  Implement an import_dbapi() classmethod directly on class <class 'pybigquery.sqlalchemy_bigquery.BigQueryDialect'> to remove this warning; the old .dbapi() classmethod may be maintained for backwards compatibility.
  engine = create_engine('bigquery://us-gcp-ame-con-52dbb-sbx-1')


In [55]:
data_frame = pd.read_sql_query(f"SELECT * FROM `input.completeness_input`",engine)
data_frame.shape

(3, 7)

In [19]:
data_frame = data_frame[data_frame.brand!='']
data_frame =data_frame.drop_duplicates(subset=["brand"])
data_frame.shape

(49, 1)

In [22]:
data_frame['title']=data_frame['title'].str.replace("&amp;", "&").replace("&quot;", '"')

In [29]:
data_frame['parent_company']=data_frame.mapply(lambda x: fetch_parent_company(x['brand'],x['title']), axis=1)

  0%|          | 0/1 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
I need to find the parent company of brand 3M
Action: Google Search
Action Input: 3M parent company
Observation: In 1951, DuPont purchased PFOA from then-Minnesota Mining and Manufacturing Company for use in the manufacturing of teflon, a product that brought DuPont a ...
Thought:I need to find the parent company of brand 3M
Action: Google Search
Action Input: 3M parent company
Observation: In 1951, DuPont purchased PFOA from then-Minnesota Mining and Manufacturing Company for use in the manufacturing of teflon, a product that brought DuPont a ...
Thought:I now know the final answer
Final Answer: Minnesota Mining and Manufacturing Company

> Finished chain.


> Entering new AgentExecutor chain...
I need to find the parent company of brand 3M
Action: Google Search
Action Input: "3M parent company"
Observation: In 1951, DuPont purchased PFOA from then-Minnesota Mining and Manufacturing Company for use in the manufacturing of teflon, a product that 

100%|██████████| 1/1 [03:58<00:00, 238.20s/it]


In [30]:
data_frame[['brand','parent_company']].value_counts()

brand  parent_company                                                           
V8     Campbell Soup Company                                                        25
ACE    Tim-Br Mart Group                                                             3
A&D    WellSpring Pharmaceutical Corporation                                         2
3M     Minnesota Mining and Manufacturing Company                                    2
Ayr    Reckitt Benckiser                                                             1
EO     EO is a brand of Young Living                                                 1
       EO Products                                                                   1
BIC    Société Bic S.A.                                                              1
BC     The parent company of BC is Goody's.                                          1
       Medtech Product Inc.                                                          1
       Brunswick Corp.                           

In [33]:
data_frame.to_csv("webscrwebscraping.csv",index=False)

In [26]:
fetch_parent_company('A&D','A&D Ointment, 16 Ounce')



> Entering new AgentExecutor chain...


I need to find the parent company behind brand A&D
Action: Google Search
Action Input: A&D parent company
Observation: A&D offers a wide and diverse range of measurement, weighing and medical equipment with analog and digital conversion as its core technology.
Thought:I need to find the parent company behind brand A&D
Action: Google Search
Action Input: A&D parent company
Observation: A&D Company, Ltd. is a world-class producer of advanced measuring, monitoring, controlling and testing instruments. Individuals with varied educational and ...
Thought:I need to find the parent company behind brand A&D
Action: Google Search
Action Input: A&D parent company
Observation: A&D Company, Ltd. is a world-class producer of advanced measuring, monitoring, controlling and testing instruments. Individuals with varied educational and ...
Thought:I now know the final answer
Final Answer: A&D Company, Ltd.

> Finished chain.


'A&D Company, Ltd.'

In [26]:
from langchain.agents import initialize_agent, Tool, load_tools, AgentType
from langchain.tools import BaseTool
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.llms import VertexAI
from langchain_community.utilities import GoogleSearchAPIWrapper
from vertexai.language_models import TextGenerationModel
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import vertexai
import os
import time
import pandas as pd
import mapply
import json
mapply.init(
    n_workers=-1,
    chunk_size=1000,
    max_chunks_per_worker=0,
    progressbar=True
)

class WebScapper:
    def __init__(self, engine):
        """
        Initializes an instance of the OutlierDetector class with the specified parameters.

        Parameters:
            engine: SQLAlchemy engine for database connection.
            schema_name (str): Name of the database schema.
            product_column (str): The column name in the DataFrame containing product information.
            output_directory (str): Directory where output files will be saved.
        """
        self.engine = engine
        self.output_directory = None
        self.target_attribute = None

    def load_data(self, schema_name, table_name, uids, target_attribute):
        """
        Retrieve rows where the specified column is NULL.

        Parameters:
            table_name (str): The name of the table.
            uids (list): List of column names used as unique identifiers.
            target_attribute (str): The column to be retrieved.
            category_column (str): The column containing category information to split the data for training & inference.

        Returns:
            pd.DataFrame: DataFrame containing training data.
        """
        self.target_attribute = target_attribute

        if self.engine:
            try:
                data = pd.read_sql_query(f"SELECT {','.join(uids)},{target_attribute} \
                                                FROM {schema_name}.{table_name} \
                                                WHERE {target_attribute} is not null and {target_attribute}!=''",
                                                self.engine)

                return data
            except Exception as e:
                print(f"Failed with error: {str(e)}")
                return {
                    'status': False,
                    'err_msg': str(e)
                }
        else:
            print('Connection Issue: Engine is not available.')
            return {
                'status': False,
                'err_msg': 'Connection Issue'
            }
    def fetch_parent_company(self, data_frame, llm, access_keys, output_directory):
        self.output_directory = output_directory
        os.environ["GOOGLE_CSE_ID"] = access_keys["GOOGLE_CSE_ID"]
        os.environ["GOOGLE_API_KEY"] = access_keys["GOOGLE_API_KEY"]

        search = GoogleSearchAPIWrapper(k=1)
        tool = [Tool(name="Google Search",
                     description="Search Google for recent results.",
                     func=search.run,
                     )]
        agent = initialize_agent(tool, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

        data_frame['parent_company'] = data_frame[self.target_attribute].apply(
            lambda brand: self.fetch_result(agent, f"What is the parent company of {brand}")
        )

        self.save_file(data_frame, "webscraper_results")

    def fetch_result(self, agent, query):
        result = agent.run(query)
        time.sleep(1)
        return result
        
    def save_file(self, df, file_name):
            """
            Saves the given DataFrame as a CSV file in the specified output directory.

            Parameters:
                df (pd.DataFrame): The DataFrame to be saved.
                file_name (str): Name of the file (without extension).
            """
            folder_path = self.output_directory
            csv_filename = f"{file_name}.csv"
            df.to_csv(f"{folder_path}/{csv_filename}", index=False)

In [24]:
data=detector.load_data(schema_name='data_quality',
                   table_name='meta_Prime_Pantry',
                   uids=['asin','title'], 
                   target_attribute='brand'
                   )

/home/user/.local/lib/python3.10/site-packages/google/cloud/bigquery/client.py:572: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


In [25]:
def fetch_parent_company(self,data_frame,llm,access_keys,output_directory):
    self.output_directory = output_directory
    os.environ["GOOGLE_CSE_ID"] = keys["GOOGLE_CSE_ID"]
    os.environ["GOOGLE_API_KEY"] = keys["GOOGLE_API_KEY"]


    search = GoogleSearchAPIWrapper(k=1)
    tool = [Tool(name="Google Search",
                description="Search Google for recent results.",
                func=search.run,
                )
            ]
    agent = initialize_agent(tool, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

    data_frame['parent_company']=data_frame[self.target_attribute].mapply(lambda x: fetch_parent_company(agent,x['brand']), axis=1)
    self.save_file(data_frame, "webscrapper_results")
    
    def fetch_parent_company(agent,brand):
        result = agent.run(f"What is the parent company of {brand}")
        time.sleep(1)
        return result

(10810, 3)

In [1]:
from langchain.agents import initialize_agent, Tool, load_tools, AgentType
from langchain.tools import BaseTool
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.llms import VertexAI
from langchain_community.utilities import GoogleSearchAPIWrapper
from vertexai.language_models import TextGenerationModel
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import vertexai
import os
import time
import pandas as pd
import mapply
import json
import logging

mapply.init(
    n_workers=-1,
    chunk_size=1000,
    max_chunks_per_worker=0,
    progressbar=True
)

class WebScrapper:
    """
    WebScrapper class for web scraping operations.

    Parameters:
        engine: SQLAlchemy engine for database connection.
    """

    def __init__(self, engine):
        self.engine = engine
        self.output_directory = None
        self.target_attribute = None

    def load_data(self, schema_name, table_name, uids, target_attribute):
        """
        Retrieve rows where the specified column is NOT NULL.

        Parameters:
            schema_name (str): Name of the database schema.
            table_name (str): The name of the table.
            uids (list): List of column names used as unique identifiers.
            target_attribute (str): The column to be retrieved.

        Returns:
            pd.DataFrame: DataFrame containing training data.
        """
        self.target_attribute = target_attribute

        if self.engine:
            try:
                query = f"SELECT {','.join(uids)},{self.target_attribute} FROM {schema_name}.{table_name} WHERE {self.target_attribute} is not null and {self.target_attribute}!=''"
                data = pd.read_sql_query(query, self.engine)
                return data
            except Exception as e:
                raise RuntimeError(f"Failed with error: {str(e)}")
        else:
            raise RuntimeError('Connection Issue: Engine is not available.')

    def fetch_parent_company(self, data_frame, llm, access_keys, output_directory):
        """
        Fetch parent company information using Google Search API.

        Parameters:
            data_frame (pd.DataFrame): DataFrame containing data.
            llm: Langchain Language Model.
            access_keys (dict): Dictionary containing Google API keys.
            output_directory (str): Directory where output files will be saved.
        """
        self.output_directory = output_directory
        os.environ["GOOGLE_CSE_ID"] = access_keys["GOOGLE_CSE_ID"]
        os.environ["GOOGLE_API_KEY"] = access_keys["GOOGLE_API_KEY"]

        search = GoogleSearchAPIWrapper(k=1)
        tool = [Tool(name="Google Search",
                     description="Search Google for recent results.",
                     func=search.run,
                     )]
        agent = initialize_agent(tool, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

        data_frame['parent_company'] = data_frame[self.target_attribute].apply(
            lambda brand: self.fetch_result(agent, f"What is the parent company of {brand}")
        )

        self.save_file(data_frame, "webscraper_results")

    def fetch_result(self, agent, query):
        """
        Fetch result using the Langchain agent.

        Parameters:
            agent: Langchain agent.
            query (str): Query to be executed.

        Returns:
            result: Result of the query execution.
        """
        try:
            result = agent.run(query)
            time.sleep(1)
            return result
        except Exception as e:
            logging.error(f"Error fetching result: {str(e)}")
            return None

    def save_file(self, df, file_name):
        """
        Saves the given DataFrame as a CSV file in the specified output directory.

        Parameters:
            df (pd.DataFrame): The DataFrame to be saved.
            file_name (str): Name of the file (without extension).
        """
        folder_path = self.output_directory
        csv_filename = f"{file_name}.csv"
        df.to_csv(f"{folder_path}/{csv_filename}", index=False)



/home/user/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from scraper import WebScraper

In [11]:
engine = create_engine('bigquery://us-gcp-ame-con-52dbb-sbx-1')

scraper = WebScraper(engine)

/tmp/ipykernel_1839/1665156092.py:1: SADeprecationWarning: The dbapi() classmethod on dialect classes has been renamed to import_dbapi().  Implement an import_dbapi() classmethod directly on class <class 'pybigquery.sqlalchemy_bigquery.BigQueryDialect'> to remove this warning; the old .dbapi() classmethod may be maintained for backwards compatibility.
  engine = create_engine('bigquery://us-gcp-ame-con-52dbb-sbx-1')


In [4]:
data=scraper.load_data(schema_name='data_quality',
                   table_name='meta_Prime_Pantry',
                   uids=['asin','title'], 
                   target_attribute='brand'
                   )
data.shape

/home/user/.local/lib/python3.10/site-packages/google/cloud/bigquery/client.py:572: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


(10810, 3)

In [5]:
data =data.drop_duplicates(subset=["brand"]).sample(n=50,random_state=42).reset_index(drop=True)
data.shape

(50, 3)

In [6]:

llm = VertexAI(
    model_name='text-bison@001',
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

/home/user/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.vertexai.VertexAI` was deprecated in langchain-community 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAI`.
  warn_deprecated(


In [7]:
with open('keys.json') as f:
    keys = json.load(f)


In [12]:
scraper.fetch_parent_company(data,llm,keys,"gs://data_quality_final/webscraping","test_record")

KeyError: None

In [1]:
wget "https://amedeloitte.sharepoint.com/:x:/r/sites/WEFPMO/_layouts/15/Doc.aspx?sourcedoc=%7BA8318C8F-4CFE-48CC-ABA9-1E069D0B702A%7D&file=sample_data_set_243001.csv&action=default&mobileredirect=true" -O sample_data_set_243001.csv

SyntaxError: invalid syntax (1068476188.py, line 1)

In [2]:
import pandas as pd
import datetime
import json
import os
from google.cloud import bigquery
import gcsfs
import mapply

mapply.init(
    n_workers=-1,
    chunk_size=100,
    max_chunks_per_worker=10,
    progressbar=True
)

client = bigquery.Client()


In [1]:

import pandas as pd
from sqlalchemy import create_engine, exc
class Test:
    def __init__(self, engine: str):
        self.engine = engine
    def cleanse_duplicate_asins_using_bq(self, df: pd.DataFrame, query: str) -> pd.DataFrame:
        """Removes rows from a DataFrame where 'asin' values are present in a BigQuery table,
        excluding duplicates from the retrieved asins.

        If the table doesn't exist, returns the original DataFrame without filtering.

        Args:
            df (pd.DataFrame): The DataFrame to filter.
            query (str): The SQL query to fetch data from BigQuery.

        Returns:
            pd.DataFrame: The filtered DataFrame, or the original DataFrame if the table doesn't exist.
        """

        try:
            asins_to_exclude = pd.read_sql_query(query, self.engine)['asin'].unique()
            return df[~df['asin'].isin(asins_to_exclude)].reset_index(drop=True)
            
        except exc.DatabaseError as e: 
            print(f"Table not found in BigQuery. Returning original DataFrame.")
            return df  # Return original DataFrame if table doesn't exist




In [2]:
engine = create_engine('bigquery://us-gcp-ame-con-52dbb-sbx-1')
exclude_scored = 'output.*'
target_attribute ='brand_parent'
source_attribute ='brand'
schema_name='test_datasets'
table_name ='sample_data_gdrive'
uids=['asin','category','title']

/tmp/ipykernel_3336/1283848504.py:1: SADeprecationWarning: The dbapi() classmethod on dialect classes has been renamed to import_dbapi().  Implement an import_dbapi() classmethod directly on class <class 'pybigquery.sqlalchemy_bigquery.BigQueryDialect'> to remove this warning; the old .dbapi() classmethod may be maintained for backwards compatibility.
  engine = create_engine('bigquery://us-gcp-ame-con-52dbb-sbx-1')


In [7]:

exclusion = f"""AND NOT EXISTS (
                SELECT {uids[0]}
                FROM `{exclude_scored}` as RESULTS
                WHERE RESULTS.TARGET_ATTRIBUTE="{target_attribute}"
                AND test_table.{uids[0]}=RESULTS.{uids[0]});"""
query = f"""SELECT {','.join(uids)},{target_attribute} AS TARGET_ATTRIBUTE_catalog_value,{source_attribute} as TARGET_ATTRIBUTE_source_attributes
        FROM {schema_name}.{table_name} AS test_table
        WHERE {source_attribute} is not null and {source_attribute}!=''
        {exclusion}"""

data = pd.read_sql_query(query, engine)
data.shape

(22, 5)

In [12]:
source_attributes = 'net_content_quantity','net_content_uom','count_per_pack','multi_quantity','total_quantity'

In [15]:
",".join(list(source_attributes))

'net_content_quantity,net_content_uom,count_per_pack,multi_quantity,total_quantity'

In [4]:
test.cleanse_duplicate_asins_using_bq(df,'select * from test_datasets.sample_data_gdrive')

,asin,category,title,description,brand,price,brand_parent,net_content_quantity,net_content_uom,count_per_pack,multi_quantity,total_quantity


In [10]:
dq=test()

TypeError: 'test' object is not callable

In [15]:
df=pd.read_csv("/home/user/ItemDataQuality/itemdataqualityrepo/csv_files/data_catalog.csv")

In [5]:
asins_to_exclude = ['B000A9K4J8', 'B000N54R5C', 'B000K54Z23']

In [16]:
df

,asin,category,title,description,brand,price,brand parent,net content quantity,net content uom,count per pack,multipack,total content,Unnamed: 12
0,B00WS3YLQ6,Prime_Pantry,"V8, Strawberry Banana, 8 Ounce, 6 Count (Pack ...","A healthy, well-balanced life. It&rsquo;s a ch...",V8,17.70000,Campbell Soup Company,8.00,oz,4.0,6.0,24.0,NaN
1,B01GCT2CJY,Prime_Pantry,"Diet Pepsi, Bottles 16.9 Fl Oz, 6 Count","With its light, crisp taste, Diet Pepsi gives ...",Pepsi,3.99000,"An ideal companion for lunch boxes, afternoon ...",16.90,FL Oz,1.0,6.0,6.0,NaN
2,B00TDFRHN6,Prime_Pantry,"BSN SYNTHA-6 Protein Powder, Strawberry Milksh...",48 Servings,BSN,40.96000,BSN,5.00,Pound,1.0,1.0,1.0,NaN
3,B010Q32SLY,Prime_Pantry,"C2O Pure Coconut Water, With Pulp, 17.5 Ounce ...",Not all coconut waters are created equal. C2O ...,C2O,NaN,"C2O Coconut Water, LLC",17.50,Ounce,1.0,12.0,12.0,NaN
4,B00OTH6HCA,Prime_Pantry,EAS Myoplex Original Ready-to-Drink Nutrition ...,EAS Myoplex Original Ready-to-Drink Protein Sh...,EAS,11.47000,Abbott Laboratories,16.90,fl oz,1.0,4.0,4.0,NaN
5,B010SI8YJW,Prime_Pantry,Os-Cal 500 D Calcium D3 200 I.U Coated Caplets...,Os-Cal 500 calcium supplement with vitamin d h...,OsCal,12.84000,GSK Plc,NaN,NaN,210.0,1.0,210.0,NaN
6,B010NBPJGU,Prime_Pantry,"Larabar Gluten Free Bars, Coconut Cream Pie, 1...","The foundation of a sound mind, body, and spir...",LRABAR,15.00000,General Mills,1.70,oz,1.0,16.0,16.0,NaN
7,B00QJGTYWO,Prime_Pantry,"Hostess Twinkies, Original, 2.7 Ounce, 6 Count",Our story begins in 1919. Thats when the world...,Hostess,6.37000,The J.M. Smucker Company,2.70,ounce,6.0,1.0,6.0,NaN
8,B00T7ATYWO,Prime_Pantry,"Trident Sugar Free Gum (Cinnamon, 18-Piece, 3-...",Get close-up confidence with Trident Sugar Fre...,Trident,10.11111,Mondelez International,NaN,NaN,18.0,3.0,54.0,NaN
9,B00V1O5X3I,Prime_Pantry,"Near East Whole Grain Blends, Roasted Pecan &a...",The delicately sweet flavor of roasted garlic ...,Quaker,2.34000,PepsiCo,5.40,oz,1.0,1.0,1.0,NaN


In [12]:
df.dtypes

no                int64
schema           object
table_name       object
args             object
error_id          int64
error_code       object
function_name    object
dtype: object

In [35]:
df.to_csv('/home/user/ItemDataQuality/itemdataqualityrepo/csv_files/validity_input.csv',index=False)

In [36]:
!gsutil cp /home/user/ItemDataQuality/itemdataqualityrepo/csv_files/validity_input.csv gs://data_quality_final/test_dataset/

Copying file:///home/user/ItemDataQuality/itemdataqualityrepo/csv_files/validity_input.csv [Content-Type=text/csv]...
/ [1 files][  242.0 B/  242.0 B]                                                
Operation completed over 1 objects/242.0 B.                                      


In [10]:
df.to_gbq(project_id = "us-gcp-ame-con-52dbb-sbx-1", destination_table = "us-gcp-ame-con-52dbb-sbx-1.test_datasets.outlier_train", if_exists='replace')


100%|██████████| 1/1 [00:00<00:00, 4739.33it/s]


In [10]:
pip install pandas_gbq

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 2.5 MB/s eta 0:00:00a 0:00:01
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.10/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.10/site-packages/p

In [51]:
from modules.multiagent import ProductInfoExtractor

In [45]:
%pwd

'/home/user/ItemDataQuality/itemdataqualityrepo'

In [47]:
from modules.outlier_detection import TextOutlierDetector

In [49]:
from modules.rag_base import RagBase

In [2]:
from sqlalchemy import *
from sqlalchemy.schema import *
from sqlalchemy.engine import create_engine
from google.cloud import bigquery
import datetime
import pandas as pd
import pandas_gbq
import logging
import string
import re
from modules.validation_module import DataValidator
from modules.dq_engine import DqEngine

In [3]:
engine = create_engine('bigquery://us-gcp-ame-con-52dbb-sbx-1')

/tmp/ipykernel_1048/111781775.py:1: SADeprecationWarning: The dbapi() classmethod on dialect classes has been renamed to import_dbapi().  Implement an import_dbapi() classmethod directly on class <class 'pybigquery.sqlalchemy_bigquery.BigQueryDialect'> to remove this warning; the old .dbapi() classmethod may be maintained for backwards compatibility.
  engine = create_engine('bigquery://us-gcp-ame-con-52dbb-sbx-1')


In [4]:
query="select * from `input.input_table_gdrive` ORDER BY input_order"

In [5]:
dataframe = pd.read_sql_query(query, engine)
dataframe

DatabaseError: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/us-gcp-ame-con-52dbb-sbx-1/queries?prettyPrint=false: Access Denied: BigQuery BigQuery: Permission denied while getting Drive credentials.
[SQL: select * from `input.input_table_gdrive` ORDER BY input_order]
(Background on this error at: https://sqlalche.me/e/20/4xp6)

In [6]:
client = bigquery.Client()

In [7]:
df = client.query(query).to_dataframe()

Forbidden: 403 Access Denied: BigQuery BigQuery: Permission denied while getting Drive credentials.; reason: accessDenied, location: 1DMFdyNH3orUvKeD7x5bTMYkqBo5gSWl-3I2xFq16GfM, message: Access Denied: BigQuery BigQuery: Permission denied while getting Drive credentials.

Location: US
Job ID: 172d7752-44d5-468d-9a47-fb7f6e1a055f
